In [1]:
import numpy as np
from functools import reduce

In [2]:
interval = [0, 1]

coordinates = [[i, j, k] for i in interval for j in interval for k in interval]
vertices = {i: coordinates[i] for i in range(len(coordinates))}

edges = {}

for index in vertices:
    if not index in edges:
        edges[index] = set()
    for other_index in vertices:
        vector = np.array(vertices[index]) - np.array(vertices[other_index])
        if np.linalg.norm(vector) == 1:
            edges[index].add(other_index)

vertices

{0: [0, 0, 0],
 1: [0, 0, 1],
 2: [0, 1, 0],
 3: [0, 1, 1],
 4: [1, 0, 0],
 5: [1, 0, 1],
 6: [1, 1, 0],
 7: [1, 1, 1]}

In [3]:
def get_pairs(several):
    pairs = []
    for x in several:
        several = several.difference({x})
        for y in several:
            pairs.append({x, y})
    return pairs

In [4]:
faces = []

remaining_vertices = set(edges.keys())
while remaining_vertices:
    v1 = remaining_vertices.pop()
    adjacent_vertices = edges[v1]
    for v2, v3 in get_pairs(adjacent_vertices):
        v4 = edges[v2].intersection(edges[v3]).difference({v1})
        face = {v1, v2, v3}.union(v4)
        faces.append(face)
    used_vertices = reduce(set.union, [edges[vertex] for vertex in adjacent_vertices]).union(adjacent_vertices)
    remaining_vertices = remaining_vertices.difference(used_vertices)

faces

[{0, 1, 2, 3},
 {0, 1, 4, 5},
 {0, 2, 4, 6},
 {1, 3, 5, 7},
 {2, 3, 6, 7},
 {4, 5, 6, 7}]

In [5]:
def on_coord_plane(face: set):
    index_list = list(face)
    coord_list = [vertices[i] for i in index_list]
    vertices_on_coord_plane = [reduce(lambda x, y: x * y, coords) == 0 for coords in coord_list]
    return True if all(vertices_on_coord_plane) else False

for face in faces:
    print(on_coord_plane(face))

True
True
True
False
False
False


In [106]:
def wind(face):
    index_list = list(face)
    coord_array = [np.array(vertices[i]) for i in index_list]
    edges = [coord_array[i % len(coord_array)] - coord_array[(i-1) % len(coord_array)] for i in range(len(coord_array))]
    v_1, v_2, v_3, v_4 = tuple(edges)
    n_1 = np.cross(v_1, v_2)
    n_2 = np.cross(v_3, v_4)
    print(coord_array)
    if np.dot(n_1, n_2) < 0:
        print(v_1)
        print(v_2)
        print(v_3)
        print(v_4)
        return "wound"
    else:
        return "not wound"